## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: GM12878
# EMBRACENET

da fare 
- sistema selezione augmentation in embracenet
- crea grafici AUPRC reti

In [1]:
import pandas as pd 
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, OrderedDict
import pickle

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# create a database to store optuna studies with sqlite backend
#engine = create_engine('sqlite:///BIOINF_optuna_tuning.db')

In [3]:
from BIOINF_tesi.data_pipe import CELL_LINES, TASKS

In [4]:
cell_line = CELL_LINES[1]
cell_line

'GM12878'

---

In [5]:
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline
from BIOINF_tesi.models import EmbraceNetMultimodal, ConcatNetMultimodal
from BIOINF_tesi.models.utils import Kfold_CV_Multimodal

In [6]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [7]:
task = TASKS[0]

In [8]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [9]:
model=EmbraceNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:38:41,853] Using an existing study with name 'GM12878_active_E_vs_inactive_E_EmbraceNetMultimodal_1' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.43202161922239113
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 64
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 128
    lr: 7.941950631839377e-05
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.8581003651107962
    weight_decay: 0.00014427681311913025

===============> MODEL TESTING
AUPRC test score: 0.43653806247169585


>>> ITERATION N. 2


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:38:42,260] Using an existing study with name 'GM12878_active_E_vs_inactive_E_EmbraceNetMultimodal_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4486517927159404
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 64
    EMBRACENET_n_units_l1: 64
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.5
    FFNN_dropout_l3: 0.5
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 16
    FFNN_n_units_l3: 4
    lr: 0.00018994441458177226
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.8728242374049127
    weight_decay: 0.00010692153089801117

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:38:42,564] Using an existing study with name 'GM12878_active_E_vs_inactive_E_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


AUPRC test score: 0.3748672128183268


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3862810869015997
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.0065169303683038325
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.10981347141895725
    weight_decay: 0.002503841704175309

===============> MODEL TESTING
AUPRC test score: 0.35868292457406403



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.39003


In [11]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [9]:
model=ConcatNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 09:20:52,445] Using an existing study with name 'GM12878_active_E_vs_inactive_E_ConcatNetMultimodal_1' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4306313393348045
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.0
    CONCATNET_dropout_l2: 0.0
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 1024
    CONCATNET_n_units_l1: 128
    CONCATNET_n_units_l2: 64
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.0043786435795655524
    optimizer: RMSprop
    weight_decay: 0.000622318648334681

===============> MODEL TESTING
AUPRC test score: 0.4292255720358358


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 09:20:53,069] Using an existing study with name 'GM12878_active_E_vs_inactive_E_ConcatNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.35871113109808417
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.2
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 1024
    CONCATNET_n_units_l1: 32
    CONCATNET_n_units_l2: 256
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.00020916590839697497
    optimizer: RMSprop
    weight_decay: 0.016118460438335157

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 09:20:53,510] Using an existing study with name 'GM12878_active_E_vs_inactive_E_ConcatNetMultimodal_1_2_3' instead of creating a new one.


AUPRC test score: 0.18309334657398224


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3476050910200529
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.0
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 1024
    CONCATNET_n_units_l1: 64
    CONCATNET_n_units_l2: 32
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 32
    lr: 0.0017723827909862963
    optimizer: RMSprop
    weight_decay: 0.00013829123871738074

===============> MODEL TESTING
AUPRC test score: 0.4098042108211864



3-FOLD CROSS-

In [11]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [9]:
model=EmbraceNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (46002) in class 0 will be larger than the number of samples in the majority class (class #0 -> 23001)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 23:08:37,676] A new study created in RDB with name: GM12878_active_E_vs_inactive_E_EmbraceNetMultimodal_1augmentation


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


EarlyStopping counter: 1 out of 4
EarlyStopping counter: 1 out of 4
EarlyStopping counter: 2 out of 4
EarlyStopping counter: 3 out of 4


...

EarlyStopping counter: 4 out of 4
Early stopping the training


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.8/site-packages/timm/optim/nadam.py:69: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1025.)
  grad = grad.add(group['weight_decay'], p.data)


EarlyStopping counter: 1 out of 4


KeyboardInterrupt: 

In [ ]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [ ]:
task = TASKS[1]
task

In [ ]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

---
### EMBRACENET

In [15]:
model=EmbraceNetMultimodal

In [16]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:38:48,795] Using an existing study with name 'GM12878_active_P_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.21729418251835664
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 32
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.3
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    lr: 0.00437196967857023
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.030424524407078657
    weight_decay: 0.0029910033153657616

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:38:49,278] Using an existing study with name 'GM12878_active_P_vs_inactive_P_EmbraceNetMultimodal_1_2' instead of creating a new one.


AUPRC test score: 0.20963793791103674


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3141121511021271
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.5
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 64
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.2
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 128
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.00992654268300641
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.46132381191053307
    weight_decay: 0.010793840912758184

===============> MODEL TESTING
AUPRC test score: 0.13951840998853346


>>> ITERATIO

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:38:49,645] Using an existing study with name 'GM12878_active_P_vs_inactive_P_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3370914852024784
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 256
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    lr: 0.0005848570978119763
    n_post_layers: 1
    optimizer: Nadam
    selection_probabilities_FFNN: 0.40907700790618196
    weight_decay: 0.0013101801416131244

===============> MODEL TESTING
AUPRC test score: 0.35296991432231567



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.23404


In [17]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [18]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [15]:
model=ConcatNetMultimodal

In [16]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 09:21:00,239] Using an existing study with name 'GM12878_active_P_vs_inactive_P_ConcatNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.20468974167824971
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.0
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 1024
    CONCATNET_n_units_l1: 64
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.4
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 16
    FFNN_n_units_l3: 32
    lr: 0.008181966656919247
    optimizer: Adam
    weight_decay: 0.02013226765271165

===============> MODEL TESTING
AUPRC test score: 0.21649410017931536


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 09:21:00,846] Using an existing study with name 'GM12878_active_P_vs_inactive_P_ConcatNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.33356500571874265
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.0
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 32
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.00010275401608951472
    optimizer: RMSprop
    weight_decay: 0.0014610045369150403

===============> MODEL TESTING
AUPRC test score: 0.3154358438817753


>>> ITERATION N. 3


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 09:21:01,219] Using an existing study with name 'GM12878_active_P_vs_inactive_P_ConcatNetMultimodal_1_2_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2947856957969999
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 96
    CONCATNET_dropout_l0: 0.5
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 512
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 16
    lr: 0.03181017468043109
    optimizer: RMSprop
    weight_decay: 0.0001780464173316517

===============> MODEL TESTING
AUPRC test score: 0.19402544532616225



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.24199


In [17]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [18]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [ ]:
model=EmbraceNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [ ]:
task = TASKS[2]
task

In [ ]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

---
### EMBRACENET

In [21]:
model=EmbraceNetMultimodal

In [22]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:38:54,788] Using an existing study with name 'GM12878_active_E_vs_active_P_EmbraceNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:38:55,207] Using an existing study with name 'GM12878_active_E_vs_active_P_EmbraceNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7759048336611933
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.5
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 64
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 128
    lr: 0.00663535768061068
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.007283701190204717
    weight_decay: 0.0006764757165046543

===============> MODEL TESTING
AUPRC test score: 0.7656655110728201


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:38:55,425] Using an existing study with name 'GM12878_active_E_vs_active_P_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


  Value:  0.8247594877279852
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.0023934027706348513
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.414316578594055
    weight_decay: 0.0073467421032437545

===============> MODEL TESTING
AUPRC test score: 0.7015546767916032


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7699332338966253
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n

In [23]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [24]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [21]:
model=ConcatNetMultimodal

In [22]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 09:21:06,237] Using an existing study with name 'GM12878_active_E_vs_active_P_ConcatNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.8263218400771184
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CONCATNET_dropout_l0: 0.3
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 512
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.0026228519175299832
    optimizer: Nadam
    weight_decay: 0.000366762230416734

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 09:21:06,480] Using an existing study with name 'GM12878_active_E_vs_active_P_ConcatNetMultimodal_1_2' instead of creating a new one.


AUPRC test score: 0.8472447574525652


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7568448778828911
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.5
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 128
    CONCATNET_dropout_l0: 0.0
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 512
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 64
    lr: 0.0024087252387324913
    optimizer: RMSprop
    weight_decay: 0.025611621335024943

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 09:21:06,706] Using an existing study with name 'GM12878_active_E_vs_active_P_ConcatNetMultimodal_1_2_3' instead of creating a new one.


AUPRC test score: 0.7825946225824127


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.8102461058393605
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 96
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.0
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 512
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 64
    lr: 0.014646452695482386
    optimizer: RMSprop
    weight_decay: 0.0002213019090405151

===============> MODEL TESTING
AUPRC test score: 0.56827674309291

In [23]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [24]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [ ]:
model=EmbraceNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [17]:
task = TASKS[3]
task

'inactive_E_vs_inactive_P'

In [18]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [27]:
model=EmbraceNetMultimodal

In [28]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:01,160] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5697440974813386
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 2.3369496029227678e-05
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.614427519073352
    weight_decay: 0.0035608847101091775

===============> MODEL TESTING
AUPRC test score: 0.5762357518981759


>>> ITERATION N. 2


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:01,935] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.580073972218564
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 64
    lr: 0.0004975144854592086
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.5611832861819466
    weight_decay: 0.0005623708010892006

===============> MODEL TESTING
AUPRC test score: 0.5806677114306839


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:02,477] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5492493027689673
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 128
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 16
    lr: 0.0004061118066150433
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.23630098920971054
    weight_decay: 0.07214613756883567

===============> MODEL TESTING
AUPRC test score: 0.5290321358741854



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.56198


In [29]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [30]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [9]:
model=ConcatNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 08:09:31,315] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_ConcatNetMultimodal_1' instead of creating a new one.
/opt/conda/lib/python3.7/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5424169979295155
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.3
    CONCATNET_dropout_l2: 0.5
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 512
    CONCATNET_n_units_l2: 16
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.4
    FFNN_dropout_l3: 0.5
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 16
    lr: 0.0002601352269233871
    optimizer: Adam
    weight_decay: 0.024930374625176378

===============> MODEL TESTING
AUPRC test score: 0.5433751994298683


>>> ITERATION N. 2


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 08:09:33,991] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_ConcatNetMultimodal_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4233390496136193
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.5
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 5
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 64
    CNN_out_channels_l3: 512
    CONCATNET_dropout_l0: 0.5
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 1024
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.01410960588607877
    optimizer: Nadam
    weight_decay: 0.018712221008761333

===============> MODEL TESTING
AUPRC test score: 0.4951939320700933


>>> ITERATION N. 3


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 08:09:34,663] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_ConcatNetMultimodal_1_2_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5700257059475923
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.5
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 64
    CONCATNET_n_units_l2: 256
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.0015129050874388707
    optimizer: Adam
    weight_decay: 0.0003263355135051064

===============> MODEL TESTING
AUPRC test score: 0.6041115455661381



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.54756


In [11]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [19]:
model=EmbraceNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [9]:
task = TASKS[4]
task

'active_EP_vs_inactive_rest'

In [ ]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

---
### EMBRACENET

In [33]:
model=EmbraceNetMultimodal

In [34]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:05,878] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.32920968060964045
  Params: 
    CNN_dropout_l0: 0.4
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.0002611315377064244
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.5831335834652084
    weight_decay: 0.002746074140090206

===============> MODEL TESTING
AUPRC test score: 0.3348131340411579


>>> ITERATION N. 2


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:06,800] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3246477522910316
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 64
    lr: 0.016943564218063267
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.666601818322622
    weight_decay: 0.0007688497723597969

===============> MODEL TESTING
AUPRC test score: 0.32828344462203274


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:07,387] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2014166149921121
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 128
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.5
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 64
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.5
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 16
    lr: 3.0744976693111935e-05
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.779615656992

In [35]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [36]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [15]:
model=ConcatNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 08:09:40,439] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_ConcatNetMultimodal_1' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.32449101101104005
  Params: 
    CNN_dropout_l0: 0.4
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.2
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 1024
    CONCATNET_n_units_l1: 512
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 5.4190802766059764e-05
    optimizer: Adam
    weight_decay: 0.0014500591114407339

===============> MODEL TESTING
AUPRC test score: 0.3244852950508097




In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [ ]:
model=EmbraceNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## FREEZED LAYERS

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

## NOT FREEZED LAYERS + AUGMENTATION

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

In [ ]:
plot_scores(cells=cell_line)